# Joint estimation of parameters

Iterative procedure to jointly fit MCAR drift parameter $\mathbf{A}$ and driving Levy parameters $(b, \Sigma)$ from $\mathbb{Y}_{\mathcal{P}} = \{\mathbf{Y}_s,\ s\in\mathcal{P}\}$.

We start by fixing $\mathbf{\hat{b}} = \mathbf{0}$ (note the choice of $\Sigma$ does not affect the estimation of $\mathbf{A}$) and:
1. Estimate $\mathbf{\hat{A}}$ using $\mathbf{\hat{b}}$;


2. Recover the increments of the background driving Levy process $\{\Delta \mathbf{L}_s (\mathbf{\hat{A}}),\ s\in\mathcal{P}\}$ using $\mathbf{\hat{A}}$ and estimate:
    
    
    - $\mathbf{\mu} := \mathbb{E}[\mathbf{L}_1]$ from $\{\Delta \mathbf{L}_s,\ s\in\mathcal{P}\}$;
    
    
    - $\Sigma$ and $\displaystyle \mathbf{f}:=\int_{\|\mathbf{z}\|>1} \mathbf{z} F(d\mathbf{z})$ from $\{\Delta \mathbf{L}_s - \hat{\mu}\Delta_s,\ s\in\mathcal{P}\}$ by applying (Gegler, 2011);
    
    
    - set $\mathbf{\hat{b}} = \mathbf{\hat\mu} - \mathbf{\hat{f}}$.


3. Repeat 1 and 2 until convergence criterion is met.

Note to estimate $\mathbf{\mu}$ we require $t$ (time horizon) to be quite large as this relies on the statistic $t^{-1} \mathbf{L}_t$ (where $\mathbf{L}_t$ is recovered...). Might be better to use a "method of moments estimator" relying on the mean of $\mathbb{Y}$ and $\mathbf{A}$ to estimate $\mathbf{\mu}$, e.g. in stationary OU process exploit the relationship $\mathbb{E}[\mathbf{Y}_s] = - \mathbf{A}^{-1} \mathbf{\mu}$ and set $\mathbf{\hat{\mu}} = - \mathbf{\hat{A}} \bar{\mathbb{Y}}_{\mathcal{P}}$.

In the following,
$$ \mathbf{\mu} = \mathbb{E}[\mathbf{L}_1] = \mathbf{b} + \int_{\|\mathbf{z}\|>1} \mathbf{z} F(d\mathbf{z}) = \mathbf{a} + \int_{\mathbb{R}^d} \mathbf{z} F(d\mathbf{z}),$$
thus 
$$ \mathbf{a} = \mathbf{b} - \int_{\|\mathbf{z}\|\leq 1} \mathbf{z} F(d\mathbf{z}).$$

In the estimation we use 
And thus for finite activity Levy processes
$$ \mathbf{\tilde{b}} = \mathbf{\mu} - \int_{\mathbb{R}^d} \mathbf{z} F(d\mathbf{z}) = \mathbf{a} = \mathbf{b} - \int_{\|\mathbf{z}\|\leq 1} \mathbf{z} F(d\mathbf{z}). $$

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from simulate_mcar import MCAR_A, gamma_increments, asymm_gamma_increments, simulate_MCAR_stat_distr_approx, simulate_MCAR_approx, simulate_MCAR_compound_poisson, simulate_MCAR_stat_distr_compound_poisson
from estimate_mcar import estimate_MCAR
import numpy as np
import scipy
from functools import partial

## OU process

In [248]:
# OU parameters
d = 2
p = 1

# build state space autoregressive matrix
AA = [np.array([[1., 0.], [0., 2.]])]
A_AA = MCAR_A(AA)

# check eigenvalues are negative, i.e. AA is in int(\mathfak{A})
evals, evecs = np.linalg.eig(A_AA)
assert (evals < 0).all()

# parameters of Levy process
b = 1*np.ones(d)
Sigma = 2*np.eye(d)

# finest grid possible
T = 10
N = 10001
P = np.linspace(0, T, N)

In [246]:
# symmetric compound poisson process
rate =10
jump_F = scipy.stats.multivariate_normal(mean=np.zeros(d), cov=np.eye(d))

np.random.seed(123)
a = b
x0 = simulate_MCAR_stat_distr_compound_poisson(A_AA, a, Sigma, rate=rate, jump_F=jump_F)
Y = simulate_MCAR_compound_poisson(P, A_AA, x0, a, Sigma, rate=rate, jump_F=jump_F, uniform=True)

b_tilde = a
AA_hat, (b_tilde_hat, Sigma_hat) = estimate_MCAR(Y, p, P, P, jump_activity='finite')

np.set_printoptions(precision=2, floatmode='fixed')
print(f'AA = \n{AA[0]}')
print(f'AA_hat = \n{AA_hat[0]}')
print('------------------------')
print(f'b_tilde = {b_tilde}')
print(f'b_tilde_hat = {b_tilde_hat}')
print('------------------------')
print(f'Sigma = \n{Sigma}')
print(f'Sigma_hat = \n{Sigma_hat}')

AA = 
[[1.00 0.00]
 [0.00 2.00]]
AA_hat = 
[[ 0.87 -0.33]
 [-0.32  2.36]]
------------------------
b_tilde = [1.00 1.00]
b_tilde_hat = [0.56 1.46]
------------------------
Sigma = 
[[2.00 0.00]
 [0.00 2.00]]
Sigma_hat = 
[[2.00e+00 1.68e-03]
 [1.68e-03 1.99e+00]]


In [252]:
class point_jumps():
    def __init__(self, d, jump_size):
        self.d = d
        self.jump_size = jump_size

    def rvs(self, size):
        return self.jump_size * np.ones((size, self.d))
    
# asymmetric compound poisson process 
rate = 2
jump_size = 2
jump_F = point_jumps(d, jump_size)

np.random.seed(123)
a = b - rate * jump_size * (np.abs(jump_size) < 1)
x0 = simulate_MCAR_stat_distr_compound_poisson(A_AA, a, Sigma, rate=rate, jump_F=jump_F)
Y = simulate_MCAR_compound_poisson(P, A_AA, x0, a, Sigma, rate=rate, jump_F=jump_F, uniform=True)

b_tilde = a
AA_hat, (b_tilde_hat, Sigma_hat) = estimate_MCAR(Y, p, P, P, jump_activity='finite')

np.set_printoptions(precision=2, floatmode='fixed')
print(f'AA = \n{AA[0]}')
print(f'AA_hat = \n{AA_hat[0]}')
print('------------------------')
print(f'b_tilde = {b_tilde}')
print(f'b_tilde_hat = {b_tilde_hat}')
print('------------------------')
print(f'Sigma = \n{Sigma}')
print(f'Sigma_hat = \n{Sigma_hat}')

AA = 
[[1.00 0.00]
 [0.00 2.00]]
AA_hat = 
[[ 1.36 -0.00]
 [-0.36  2.16]]
------------------------
b_tilde = [1.00 1.00]
b_tilde_hat = [2.99 0.28]
------------------------
Sigma = 
[[2.00 0.00]
 [0.00 2.00]]
Sigma_hat = 
[[ 2.04 -0.04]
 [-0.04  2.00]]


In [253]:
# symmetric Gamma increments
jumps = partial(gamma_increments, shape=5, scale=5, d=d)

np.random.seed(123)
a = b
x0 = simulate_MCAR_stat_distr_approx(A_AA, a, Sigma, jumps)
Y = simulate_MCAR_approx(P, A_AA, x0, a, Sigma, jumps, uniform=True)

AA_hat, (b_hat, Sigma_hat) = estimate_MCAR(Y, p, P, P, jump_activity = 'infinite')

np.set_printoptions(precision=2, floatmode='fixed')
print(f'AA = \n{AA[0]}')
print(f'AA_hat = \n{AA_hat[0]}')
print('------------------------')
print(f'b = {b}')
print(f'b_hat = {b_hat}')
print('------------------------')
print(f'Sigma = \n{Sigma}')
print(f'Sigma_hat = \n{Sigma_hat}')

AA = 
[[1.00 0.00]
 [0.00 2.00]]
AA_hat = 
[[ 0.92  0.14]
 [-0.06  2.13]]
------------------------
b = [1.00 1.00]
b_hat = [2.53 1.83]
------------------------
Sigma = 
[[2.00 0.00]
 [0.00 2.00]]
Sigma_hat = 
[[2.17 0.02]
 [0.02 2.13]]


In [260]:
# asymmetric Gamma increments
jumps = partial(asymm_gamma_increments, shape=1, scale=1, d=d)

np.random.seed(123)
a = b - (1 - np.exp(-1))
x0 = simulate_MCAR_stat_distr_approx(A_AA, a, Sigma, jumps)
Y = simulate_MCAR_approx(P, A_AA, x0, a, Sigma, jumps, uniform=True)

AA_hat, (b_hat, Sigma_hat) = estimate_MCAR(Y, p, P, P, jump_activity='infinite')

np.set_printoptions(precision=2, floatmode='fixed')
print(f'AA = \n{AA[0]}')
print(f'AA_hat = \n{AA_hat[0]}')
print('------------------------')
print(f'b = {b}')
print(f'b_hat = {b_hat}')
print('------------------------')
print(f'Sigma = \n{Sigma}')
print(f'Sigma_hat = \n{Sigma_hat}')

AA = 
[[1.00 0.00]
 [0.00 2.00]]
AA_hat = 
[[0.77 0.05]
 [0.12 2.94]]
------------------------
b = [1.00 1.00]
b_hat = [1.08 1.10]
------------------------
Sigma = 
[[2.00 0.00]
 [0.00 2.00]]
Sigma_hat = 
[[2.03 0.05]
 [0.05 1.97]]
